In [8]:
import numpy as np
import pandas as pd
import datetime

import cmocean
import folium

def cmocean_to_leaflet(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, "#{:02x}{:02x}{:02x}".format(C[0], C[1], C[2])])
        
    return pl_colorscale

thermal = cmocean_to_leaflet(cmocean.cm.thermal, 1000)

def rgb2hex(r,g,b):
    hex = "#{:02x}{:02x}{:02x}".format(r,g,b)
    return hex

def color_normalize(cmin,cmax,value):
    return (value-cmin)/(cmax-cmin)

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

tol = .00001


In [9]:
class ARGOS_SERVICE(object):
    r"""Download and parse data from the University of Wyoming's upper air archive."""

    @staticmethod
    def get_data(filename=None, ARGOS_Type=None):
        r"""
        Basic Method to open files.  Specific actions can be passes as kwargs for instruments
        """

        fobj = open(filename)
        data = fobj.read()


        buf = data
        return BytesIO(buf.strip())

    @staticmethod
    def drifter_parse(fobj):
        r"""

        """
        argo_to_datetime =lambda date: datetime.datetime.strptime(date, '%Y %j %H%M')

        header=['argosid','lat','lon','year','doy','hhmm','s1','s2','s3','s4','s5','s6','s7','s8']
        df = pd.read_csv(fobj,delimiter='\s+',header=0,
          names=header,index_col=False,
          dtype={'year':str,'doy':str,'hhmm':str,'s1':str,'s2':str,'s3':str,'s4':str,'s5':str,'s6':str,'s7':str,'s8':str},
          parse_dates=[['year','doy','hhmm']],date_parser=argo_to_datetime)

        df.set_index(pd.DatetimeIndex(df['year_doy_hhmm']),inplace=True)
        df.drop('year_doy_hhmm',axis=1,inplace=True)

        return df

def sst_argos(s1,s2):
    try:
        output = int(format(int(s1,16),'08b')[6:] + format(int(s2,16),'08b'),2) 
        output = (output * 0.04) - 2.0   
    except:
        output = 1e35
    return output

def strain_argos(s1,manufacter='MetOcean'):
    try:
      converted_word = int(format(int(s1,16),'08b'),2)
      if manufacter == 'MetOcean':
        output = converted_word
      else:
        output = converted_word / 100.
    except:
      output = 1e35
    return output

def voltage_argos(s1):
    try:
        converted_word = int(format(int(s1,16),'08b')[:6],2)
        output = (converted_word * 0.2) + 5   
    except:
        output = 1e35
    return output

def checksum_argos(s1,s2,s3,s4):
    try:
        converted_word = int(format(int(s1,16),'08b'),2) + \
                         int(format(int(s2,16),'08b'),2) + \
                         int(format(int(s3,16),'08b'),2)
        checksum_test = converted_word % 256 
        if  checksum_test == int(format(int(s4,16),'08b'),2):
          output = True
        else:
          output = False
    except:
        output = 1e35
    return output

In [47]:
"""--------------------- Get Data  -----------------------------------------"""
## Data is read in from raw drifter id files
argo_to_datetime =lambda date: datetime.datetime.strptime(date, '%Y %j %H%M')

header=['argosid','lat','lon','year','doy','hhmm','s1','s2','s3','s4','s5','s6','s7','s8']
df_122537 = pd.read_csv('data/drifters/122537.y2017',delimiter='\s+',header=0,
  names=header,index_col=False,
  dtype={'year':str,'doy':str,'hhmm':str,'s1':str,'s2':str,'s3':str,'s4':str,'s5':str,'s6':str,'s7':str,'s8':str},
  parse_dates=[['year','doy','hhmm']],date_parser=argo_to_datetime,error_bad_lines=False)

df_122537.set_index(pd.DatetimeIndex(df_122537['year_doy_hhmm']),inplace=True)
df_122537.drop('year_doy_hhmm',axis=1,inplace=True)

df_122537['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df_122537.index]
df_122537['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df_122537.index]
df_122537['doyfrac'] = df_122537['doy'] + df_122537['fracday']


##
df_122541 = pd.read_csv('data/drifters/122541.y2017',delimiter='\s+',header=0,
  names=header,index_col=False,
  dtype={'year':str,'doy':str,'hhmm':str,'s1':str,'s2':str,'s3':str,'s4':str,'s5':str,'s6':str,'s7':str,'s8':str},
  parse_dates=[['year','doy','hhmm']],date_parser=argo_to_datetime,error_bad_lines=False)

df_122541.set_index(pd.DatetimeIndex(df_122541['year_doy_hhmm']),inplace=True)
df_122541.drop('year_doy_hhmm',axis=1,inplace=True)

df_122541['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df_122541.index]
df_122541['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df_122541.index]
df_122541['doyfrac'] = df_122541['doy'] + df_122541['fracday']

##
df_122542 = pd.read_csv('data/drifters/122542.y2017',delimiter='\s+',header=0,
  names=header,index_col=False,
  dtype={'year':str,'doy':str,'hhmm':str,'s1':str,'s2':str,'s3':str,'s4':str,'s5':str,'s6':str,'s7':str,'s8':str},
  parse_dates=[['year','doy','hhmm']],date_parser=argo_to_datetime,error_bad_lines=False)

df_122542.set_index(pd.DatetimeIndex(df_122542['year_doy_hhmm']),inplace=True)
df_122542.drop('year_doy_hhmm',axis=1,inplace=True)

df_122542['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df_122542.index]
df_122542['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df_122542.index]
df_122542['doyfrac'] = df_122542['doy'] + df_122542['fracday']

##
df_136868 = pd.read_csv('data/drifters/136868.y2017',delimiter='\s+',header=0,
  names=header,index_col=False,
  dtype={'year':str,'doy':str,'hhmm':str,'s1':str,'s2':str,'s3':str,'s4':str,'s5':str,'s6':str,'s7':str,'s8':str},
  parse_dates=[['year','doy','hhmm']],date_parser=argo_to_datetime,error_bad_lines=False)

df_136868.set_index(pd.DatetimeIndex(df_136868['year_doy_hhmm']),inplace=True)
df_136868.drop('year_doy_hhmm',axis=1,inplace=True)

df_136868['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df_136868.index]
df_136868['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df_136868.index]
df_136868['doyfrac'] = df_136868['doy'] + df_136868['fracday']

##
df_136869 = pd.read_csv('data/drifters/136869.y2017',delimiter='\s+',header=0,
  names=header,index_col=False,
  dtype={'year':str,'doy':str,'hhmm':str,'s1':str,'s2':str,'s3':str,'s4':str,'s5':str,'s6':str,'s7':str,'s8':str},
  parse_dates=[['year','doy','hhmm']],date_parser=argo_to_datetime,error_bad_lines=False)

df_136869.set_index(pd.DatetimeIndex(df_136869['year_doy_hhmm']),inplace=True)
df_136869.drop('year_doy_hhmm',axis=1,inplace=True)

df_136869['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df_136869.index]
df_136869['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df_136869.index]
df_136869['doyfrac'] = df_136869['doy'] + df_136869['fracday']


Skipping line 6759: expected 14 fields, saw 38



In [55]:
### resample to fixed time grid and fill interpolated temperatures with large value

df_122537 = df_122537.resample('12H').mean()
df_122537=df_122537.interpolate()

f1 = folium.map.FeatureGroup()

lats = df_122537['lat'].values
lngs = df_122537['lon'].values

for lat, lng in zip(lats, lngs):
    f1.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=.1,
            color='#000000',
        )
        )

In [56]:
### resample to fixed time grid and fill interpolated temperatures with large value

df_122542 = df_122542.resample('12H').mean()
df_122542=df_122542.interpolate()

f2 = folium.map.FeatureGroup()

lats = df_122542['lat'].values
lngs = df_122542['lon'].values

for lat, lng in zip(lats, lngs):
    f2.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=.1,
            color='#000000',
        )
        )

In [57]:
### resample to fixed time grid and fill interpolated temperatures with large value

df_122541 = df_122541.resample('12H').mean()
df_122541=df_122541.interpolate()

f3 = folium.map.FeatureGroup()

lats = df_122541['lat'].values
lngs = df_122541['lon'].values

for lat, lng in zip(lats, lngs):
    f3.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=.1,
            color='#000000',
        )
        )

In [59]:
### resample to fixed time grid and fill interpolated temperatures with large value

df_136868 = df_136868.resample('12H').mean()
df_136868 = df_136868.interpolate()

f4 = folium.map.FeatureGroup()

lats = df_136868['lat'].values
lngs = df_136868['lon'].values

for lat, lng in zip(lats, lngs):
    f4.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=.1,
            color='#000000',
        )
        )

In [60]:
### resample to fixed time grid and fill interpolated temperatures with large value

df_136869 = df_136869.resample('12H').mean()
df_136869 = df_136869.interpolate()

f5 = folium.map.FeatureGroup()

lats = df_136869['lat'].values
lngs = df_136869['lon'].values

for lat, lng in zip(lats, lngs):
    f5.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=.1,
            color='#000000',
        )
        )

In [61]:
m = folium.Map(location=[71,-160],
              tiles='https://api.mapbox.com/styles/v1/sbell/cj48p13bt265k2qvucuielhm7/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1Ijoic2JlbGwiLCJhIjoiY2lqbGlpaHBkMDAyanV5bHhqMTdjYTd5aiJ9.04PwNcY3Piny-YtIg5cIJA',
              attr='North Star - MapBox',
              zoom_start=6)
m.add_child(f1)
m.add_child(f2)
m.add_child(f3)
m.add_child(f4)
m.add_child(f5)

